In [ ]:
import pandas as pd 

df = pd.read_excel("input/S1File.xlsx")

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-paper')

def save_fig(name):
    plt.savefig(f'figures/{name}.png', dpi=300, facecolor='white', transparent=False)

In [ ]:
# Count successful and unsuccessful grafts
graft_counts = df['Graftfailure'].value_counts()
successful_count = graft_counts[0]  # Graftfailure = 0 means successful
unsuccessful_count = graft_counts[1]  # Graftfailure = 1 means unsuccessful

# Create bar plot
plt.figure(figsize=(8, 6))
categories = ['Successful Grafts', 'Unsuccessful Grafts']
counts = [successful_count, unsuccessful_count]

plt.bar(categories, counts, color=['green', 'red'], alpha=0.7)
plt.title('Number of Successful vs Unsuccessful Grafts')
plt.ylabel('Number of Patients')
plt.xlabel('Graft Outcome')

# Add value labels on top of bars
for i, count in enumerate(counts):
    plt.text(i, count + 4, str(count), ha='center', va='bottom')

plt.tight_layout()
save_fig('graft_outcome_distribution')
plt.show()

In [ ]:
successful_graft_df = df[df['Graftfailure'] != 1]

successful_graft_df

In [ ]:
# Create mosaic plot for demographic distributions
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Demographic Distributions within Successful Grafts', fontsize=16)

# Plot GE0DER (Gender)
gender_counts = successful_graft_df['GE0DER'].value_counts()
gender_labels = ['Male' if x == 1 else 'Female' for x in gender_counts.index]
axes[0, 0].pie(gender_counts.values, labels=gender_labels, autopct='%1.1f%%')
axes[0, 0].set_title('Gender Distribution')

# Plot BMI distribution
axes[0, 1].hist(successful_graft_df['BMI'].dropna(), bins=20, alpha=0.7, color='skyblue')
axes[0, 1].set_title('BMI Distribution')
axes[0, 1].set_xlabel('BMI')
axes[0, 1].set_ylabel('Frequency')

# Plot Age distribution
axes[0, 2].hist(successful_graft_df['Age(1)'].dropna(), bins=20, alpha=0.7, color='lightgreen')
axes[0, 2].set_title('Age Distribution')
axes[0, 2].set_xlabel('Age')
axes[0, 2].set_ylabel('Frequency')

# Plot KT_TYPE
kt_type_counts = successful_graft_df['KT_TYPE'].value_counts()
kt_type_labels = ['Living Donor' if x == 1 else 'Deceased Donor' for x in kt_type_counts.index]
axes[1, 0].pie(kt_type_counts.values, labels=kt_type_labels, autopct='%1.1f%%')
axes[1, 0].set_title('Kidney Transplant Type')

# Plot DONOR_TYPE
donor_type_counts = successful_graft_df['DONOR_TYPE'].value_counts()
axes[1, 1].bar(donor_type_counts.index, donor_type_counts.values, color='orange', alpha=0.7)
axes[1, 1].set_title('Donor Type Distribution')
axes[1, 1].set_xlabel('Donor Type')
axes[1, 1].set_ylabel('Count')

# Remove the empty subplot
axes[1, 2].remove()

plt.tight_layout()
save_fig('demographics')
plt.show()

In [ ]:
import numpy as np

# Identify numerical and categorical columns in X
numerical_columns = successful_graft_df.select_dtypes(include=[np.number]).columns.tolist()
categorical_columns = successful_graft_df.select_dtypes(include=['object']).columns.tolist()

print("Numerical columns:")
print(numerical_columns)
print(f"\nNumber of numerical columns: {len(numerical_columns)}")

print("\nCategorical columns:")
print(categorical_columns)
print(f"\nNumber of categorical columns: {len(categorical_columns)}")


In [ ]:
# Count missing values per row
missing_counts = successful_graft_df.isnull().sum(axis=1)

# Find the row with the minimum number of missing values
min_missing_idx = missing_counts.idxmin()
min_missing_count = missing_counts.min()

print(f"Row with least missing values: Index {min_missing_idx}")
print(f"Number of missing values: {min_missing_count}")
print("\nRow data:")
print(dict(successful_graft_df.loc[min_missing_idx]))

In [ ]:

X = successful_graft_df[[
    'INITIAL_MPA', 'INITIAL_MAINIS', 'INITIAL_CNI', 'DONOR_TYPE', 'PRA2', 'PRA1', 'Age(1)', 
    'HEIGHT', 'WEIGHT', 'eGFR_DC', 'HLA_MN', 'KT_TYPE', 'sCR_DC', 'INDUCTION_TYPE', 
    'INDUCTION_YN', 'GE0DER'
]]
y = successful_graft_df['MPA_dose']

str_columns = [col for col in X.columns if X[col].dtype == 'object']
print("Columns with string (object) data:")
print(str_columns)

for col in X.columns:
    X[col] = (X[col].map(lambda x: float(str(x).split(" ")[0].split("(")[0])))

X.to_csv('output/X.csv', index=False)

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import pandas as pd

# Create an IterativeImputer for multiple imputation
imputer = IterativeImputer(random_state=42, max_iter=10)

# Fit and transform the data
X_imputed = imputer.fit_transform(X)

# Convert back to DataFrame with original column names
X_imputed = pd.DataFrame(X_imputed, columns=X.columns, index=X.index)

print("Multiple imputation completed!")
print(f"Original missing values: {X.isnull().sum().sum()}")
print(f"Missing values after imputation: {X_imputed.isnull().sum().sum()}")

# Display comparison of before and after for columns with missing values
missing_cols = X.columns[X.isnull().any()].tolist()
if missing_cols:
    print(f"\nColumns that had missing values: {missing_cols}")
    for col in missing_cols[:5]:  # Show first 5 columns with missing values
        print(f"\n{col}:")
        print(f"  Original mean: {X[col].mean():.2f}")
        print(f"  Imputed mean: {X_imputed[col].mean():.2f}")

In [ ]:
import numpy as np
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# 1. Correlation Analysis
plt.figure(figsize=(12, 10))

correlation_matrix = X.corr()

# Create correlation heatmap
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', center=0, square=True, linewidths=0.5)
plt.title('Correlation Matrix Between All Variables within Successful Grafts')
plt.tight_layout()
save_fig('correlation_matrix')
plt.show()

# 2. MPA dosage analysis by Kidney Type and Donor Type
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# MPA dosage by Donor Type
donor_type_labels = {1: 'Living Donor', 2: 'Deceased Donor'}
successful_graft_df['Donor_Label'] = successful_graft_df['DONOR_TYPE'].map(donor_type_labels)

axes[0].boxplot([successful_graft_df[successful_graft_df['DONOR_TYPE']==1]['MPA_dose'],
                 successful_graft_df[successful_graft_df['DONOR_TYPE']==2]['MPA_dose']], 
                labels=['Living Donor', 'Deceased Donor'])
axes[0].set_title('MPA Dosage Distribution by Donor Type within Successful Grafts')
axes[0].set_ylabel('MPA Dosage (mg)')
axes[0].set_xlabel('Donor Type')

# MPA dosage by Kidney Type
kt_type_labels = {1: 'Living Donor KT', 2: 'Deceased Donor KT'}
successful_graft_df['KT_Label'] = successful_graft_df['KT_TYPE'].map(kt_type_labels)

axes[1].boxplot([successful_graft_df[successful_graft_df['KT_TYPE']==1]['MPA_dose'],
                 successful_graft_df[successful_graft_df['KT_TYPE']==2]['MPA_dose']], 
                labels=['Living Donor KT', 'Deceased Donor KT'])
axes[1].set_title('MPA Dosage Distribution by Kidney Type within Successful Grafts')
axes[1].set_ylabel('MPA Dosage (mg)')
axes[1].set_xlabel('Kidney Type')

plt.tight_layout()
save_fig('mpa_dosage_analysis')
plt.show()

# Print dosage statistics
print("MPA Dosage Statistics by Donor Type:")
for donor_type in [1, 2]:
    subset = successful_graft_df[successful_graft_df['DONOR_TYPE'] == donor_type]
    label = donor_type_labels[donor_type]
    mean_dose = subset['MPA_dose'].mean()
    print(f"{label}: Mean = {mean_dose:.1f} mg")

print("\nMPA Dosage Statistics by Kidney Type:")
for kt_type in [1, 2]:
    subset = successful_graft_df[successful_graft_df['KT_TYPE'] == kt_type]
    label = kt_type_labels[kt_type]
    mean_dose = subset['MPA_dose'].mean()
    print(f"{label}: Mean = {mean_dose:.1f} mg")

In [ ]:
model_stats = {}

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error

rf = RandomForestRegressor(n_estimators=250, random_state=42, criterion='squared_error')
cv_scores_rf = cross_val_score(rf, X_imputed, y, cv=5, scoring='neg_mean_squared_error')
cv_scores_mae_rf = cross_val_score(rf, X, y, cv=5, scoring='neg_mean_absolute_error')
cv_scores_r2_rf = cross_val_score(rf, X_imputed, y, cv=5, scoring='r2')

model_stats['Random Forest'] = {
    'mse_mean': -cv_scores_rf.mean(),
    'mse_std': cv_scores_rf.std(),
    'mae_mean': -cv_scores_mae_rf.mean(),
    'mae_std': cv_scores_mae_rf.std(),
    'r2_mean': cv_scores_r2_rf.mean(),
    'r2_std': cv_scores_r2_rf.std()
}

print("5-Fold CV MSE (mean ± std): {:.2f} ± {:.2f}".format(-cv_scores_rf.mean(), cv_scores_rf.std()))
print("5-Fold CV MAE (mean ± std): {:.2f} ± {:.2f}".format(-cv_scores_mae_rf.mean(), cv_scores_mae_rf.std()))
print("5-Fold CV R2 (mean ± std): {:.2f} ± {:.2f}".format(cv_scores_r2_rf.mean(), cv_scores_rf.std()))

In [ ]:
from xgboost import XGBRegressor

# XGBoost Regressor

xgb = XGBRegressor(n_estimators=250, random_state=42, n_jobs=-1)
cv_scores_xgb = cross_val_score(xgb, X_imputed, y, cv=5, scoring='neg_mean_squared_error')
cv_scores_mae_xgb = cross_val_score(xgb, X_imputed, y, cv=5, scoring='neg_mean_absolute_error')
cv_scores_r2_xgb = cross_val_score(xgb, X_imputed, y, cv=5, scoring='r2')

model_stats['XGBoost'] = {
    'mse_mean': -cv_scores_xgb.mean(),
    'mse_std': cv_scores_xgb.std(),
    'mae_mean': -cv_scores_mae_xgb.mean(),
    'mae_std': cv_scores_mae_xgb.std(),
    'r2_mean': cv_scores_r2_xgb.mean(),
    'r2_std': cv_scores_r2_xgb.std()
}

print("XGBoost Results:")
print("5-Fold CV MSE (mean ± std): {:.2f} ± {:.2f}".format(-cv_scores_xgb.mean(), cv_scores_xgb.std()))
print("5-Fold CV MAE (mean ± std): {:.2f} ± {:.2f}".format(-cv_scores_mae_xgb.mean(), cv_scores_mae_xgb.std()))
print("5-Fold CV R2 (mean ± std): {:.2f} ± {:.2f}".format(cv_scores_r2_xgb.mean(), cv_scores_r2_xgb.std()))

In [ ]:
from catboost import CatBoostRegressor

# CatBoost Regressor
catboost = CatBoostRegressor(iterations=250, random_seed=42, verbose=False)
cv_scores_catboost = cross_val_score(catboost, X_imputed, y, cv=5, scoring='neg_mean_squared_error')
cv_scores_mae_catboost = cross_val_score(catboost, X_imputed, y, cv=5, scoring='neg_mean_absolute_error')
cv_scores_r2_catboost = cross_val_score(catboost, X_imputed, y, cv=5, scoring='r2')

model_stats['CatBoost'] = {
    'mse_mean': -cv_scores_catboost.mean(),
    'mse_std': cv_scores_catboost.std(),
    'mae_mean': -cv_scores_mae_catboost.mean(),
    'mae_std': cv_scores_mae_catboost.std(),
    'r2_mean': cv_scores_r2_catboost.mean(),
    'r2_std': cv_scores_r2_catboost.std()
}

print("CatBoost Results:")
print("5-Fold CV MSE (mean ± std): {:.2f} ± {:.2f}".format(-cv_scores_catboost.mean(), cv_scores_catboost.std()))
print("5-Fold CV MAE (mean ± std): {:.2f} ± {:.2f}".format(-cv_scores_mae_catboost.mean(), cv_scores_mae_catboost.std()))
print("5-Fold CV R2 (mean ± std): {:.2f} ± {:.2f}".format(cv_scores_r2_catboost.mean(), cv_scores_r2_catboost.std()))

In [ ]:
from lightgbm import LGBMRegressor

# LightGBM Regressor
lgbm = LGBMRegressor(n_estimators=250, random_state=42, n_jobs=-1)
cv_scores_lgbm = cross_val_score(lgbm, X_imputed, y, cv=5, scoring='neg_mean_squared_error')
cv_scores_mae_lgbm = cross_val_score(lgbm, X_imputed, y, cv=5, scoring='neg_mean_absolute_error')
cv_scores_r2_lgbm = cross_val_score(lgbm, X_imputed, y, cv=5, scoring='r2')

model_stats['LightGBM'] = {
    'mse_mean': -cv_scores_lgbm.mean(),
    'mse_std': cv_scores_lgbm.std(),
    'mae_mean': -cv_scores_mae_lgbm.mean(),
    'mae_std': cv_scores_mae_lgbm.std(),
    'r2_mean': cv_scores_r2_lgbm.mean(),
    'r2_std': cv_scores_r2_lgbm.std()
}

print("LightGBM Results:")
print("5-Fold CV MSE (mean ± std): {:.2f} ± {:.2f}".format(-cv_scores_lgbm.mean(), cv_scores_lgbm.std()))
print("5-Fold CV MAE (mean ± std): {:.2f} ± {:.2f}".format(-cv_scores_mae_lgbm.mean(), cv_scores_mae_lgbm.std()))
print("5-Fold CV R2 (mean ± std): {:.2f} ± {:.2f}".format(cv_scores_r2_lgbm.mean(), cv_scores_r2_lgbm.std()))

In [ ]:
from gplearn.genetic import SymbolicRegressor
from sklearn.model_selection import cross_val_score

# Train a Symbolic Regressor from gplearn
gplearn_reg = SymbolicRegressor(
    population_size=1000, generations=20, stopping_criteria=0.01,
    p_crossover=0.7, p_subtree_mutation=0.1, p_hoist_mutation=0.05,
    p_point_mutation=0.1, max_samples=0.9, verbose=1, random_state=42, n_jobs=-1
)

gplearn_reg.fit(X_imputed, y)

# Cross-validation scores
cv_scores_gplearn = cross_val_score(gplearn_reg, X_imputed, y, cv=5, scoring='neg_mean_squared_error')
cv_scores_mae_gplearn = cross_val_score(gplearn_reg, X_imputed, y, cv=5, scoring='neg_mean_absolute_error')
cv_scores_r2_gplearn = cross_val_score(gplearn_reg, X_imputed, y, cv=5, scoring='r2')

model_stats['SymbolicRegressor'] = {
    'mse_mean': -cv_scores_gplearn.mean(),
    'mse_std': cv_scores_gplearn.std(),
    'mae_mean': -cv_scores_mae_gplearn.mean(),
    'mae_std': cv_scores_mae_gplearn.std(),
    'r2_mean': cv_scores_r2_gplearn.mean(),
    'r2_std': cv_scores_r2_gplearn.std()
}

print("Symbolic Regressor training complete.")

In [ ]:
model_stats_df = pd.DataFrame(model_stats).T

model_stats_df.to_csv('output/model_stats.csv')

model_stats_df

In [ ]:
# Create a comprehensive model comparison plot
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Plot 1: MSE Comparison
models = model_stats_df.index
mse_means = model_stats_df['mse_mean']
mse_stds = model_stats_df['mse_std']

axes[0].bar(models, mse_means, yerr=mse_stds, capsize=5, alpha=0.7, color='skyblue')
axes[0].set_title('Model Comparison: Mean Squared Error')
axes[0].set_ylabel('MSE')
axes[0].tick_params(axis='x')
axes[0].grid(axis='y', alpha=0.3)

# Plot 2: MAE Comparison
mae_means = model_stats_df['mae_mean']
mae_stds = model_stats_df['mae_std']

axes[1].bar(models, mae_means, yerr=mae_stds, capsize=5, alpha=0.7, color='lightcoral')
axes[1].set_title('Model Comparison: Mean Absolute Error')
axes[1].set_ylabel('MAE')
axes[1].tick_params(axis='x')
axes[1].grid(axis='y', alpha=0.3)

# Plot 3: R² Score Comparison
r2_means = model_stats_df['r2_mean']
r2_stds = model_stats_df['r2_std']

axes[2].bar(models, r2_means, yerr=r2_stds, capsize=5, alpha=0.7, color='lightgreen')
axes[2].set_title('Model Comparison: R² Score')
axes[2].set_ylabel('R² Score')
axes[2].tick_params(axis='x')
axes[2].grid(axis='y', alpha=0.3)

plt.tight_layout()
save_fig('model_comparison')
plt.show()

In [ ]:
import shap

# Prepare models and names
models = [
    ('Random Forest', rf),
    ('XGBoost', xgb),
    ('CatBoost', catboost),
    ('LightGBM', lgbm),
    ('SymbolicRegressor', gplearn_reg)
]

X_display = X_imputed.copy()

# Fit all models (if not already fitted)
for name, model in models:
    if not hasattr(model, 'feature_importances_') and name != 'SymbolicRegressor':
        model.fit(X_display, y)
    elif name == 'SymbolicRegressor' and not hasattr(model, '_program'):
        model.fit(X_display, y)

# Prepare SHAP explainers and plot summary plots
for name, model in models:
    print(f"Generating SHAP summary plot for {name}...")
    plt.figure(figsize=(10, 6))
    
    if name == 'SymbolicRegressor':
        # Use Permutation explainer for SymbolicRegressor
        explainer = shap.Explainer(model.predict, X_display)
        shap_values = explainer(X_display)
        shap.summary_plot(shap_values, X_display, feature_names=X_display.columns, plot_type="bar", max_display=10, show=False)
    elif name in ['CatBoost', 'LightGBM', 'Random Forest', 'XGBoost']:
        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(X_display)
        shap.summary_plot(shap_values, X_display, feature_names=X_display.columns, plot_type="bar", max_display=10, show=False)
    
    plt.title(f'SHAP Feature Importance - {name}')
    save_fig(f'shap_{name.lower().replace(" ", "_")}')
    plt.show()
